In [3]:
import pandas as pd
df = pd.read_excel('/Volumes/Air External/Downloads/curd to by ptoduct (1).xlsx')
df.head(5)

<ipython-input-3-f6f795fc21fa>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df = pd.read_excel('/Volumes/Air External/Downloads/curd to by ptoduct (1).xlsx')


,SL.No,Booth Code,by product code,Zone,First Name,Last Name,Mobile
0,1,9003,33012,EAST,R,Gopalakrishnan,9488160693
1,4,9002,82026,WEST,NANDHINIDEVI,G,0
2,6,9000,82041,SOUTH,MANIVANNAN,V,1952455505
3,7,9011,99819,SOUTH,KP MOHAMMED RIYAZ,NaN,9943166659
4,9,9019,99883,NORTH,L,Rajkumar,0


In [5]:
for index, row in df.iterrows():
    if Business.objects.filter(code=row['Booth Code']).exists():
        business_obj = Business.objects.get(code=row['Booth Code'])
        business_obj.code = row['by product code']
        business_obj.save()
        print(f"Booth code updated for {row['by product code']}")
        user_obj = User.objects.get(id=business_obj.user_profile.user.id)
        user_obj.username = row['by product code']
        user_obj.save()
        print('-----------------------------------------------------')
        print('User name updated')

Booth code updated for 33012
-----------------------------------------------------
User name updated
Booth code updated for 82026
-----------------------------------------------------
User name updated
Booth code updated for 82041
-----------------------------------------------------
User name updated
Booth code updated for 99819
-----------------------------------------------------
User name updated
Booth code updated for 99883
-----------------------------------------------------
User name updated
Booth code updated for 99767
-----------------------------------------------------
User name updated
Booth code updated for 99760
-----------------------------------------------------
User name updated
Booth code updated for 99801
-----------------------------------------------------
User name updated
Booth code updated for 10088
-----------------------------------------------------
User name updated
Booth code updated for 99876
-----------------------------------------------------
User nam

In [10]:
def findout_tray_count(count, product_defalut_count, product_id):
    loose_milk_ids = [22, 23, 24]
    if not product_defalut_count == 0:
        remainder_of_tray_count = count % product_defalut_count
        coefieient_of_tray_count = count // product_defalut_count
    else:
        remainder_of_tray_count = 0
        coefieient_of_tray_count = 0
    if not remainder_of_tray_count == 0:
        if not product_id in loose_milk_ids:
            if remainder_of_tray_count > (product_defalut_count / 2):
                return coefieient_of_tray_count + 1
            else:
                return coefieient_of_tray_count
        else:
            if remainder_of_tray_count > 0:
                return coefieient_of_tray_count + 1
            else:
                return coefieient_of_tray_count
    else:
        return coefieient_of_tray_count

In [19]:
def check_tray_count(route_id, date, session_id):
    route = Route.objects.get(id=route_id)
    business_ids = RouteBusinessMap.objects.filter(route=route).values_list('business', flat=True)
    sale_group = SaleGroup.objects.filter(business_id__in=business_ids, date=date, session_id=session_id)
    sale_values = Sale.objects.filter(sale_group__in=sale_group, product__is_active=True).values_list('product_id',
                                                                                                      'count')
    sale_columns = ['product_id', 'count']
    sale_df = pd.DataFrame(list(sale_values), columns=sale_columns)
    icustomer_sale_group = ICustomerSaleGroup.objects.filter(business_id__in=business_ids, date__month=date.month,
                                                             date__year=date.year, session_id=session_id)
    icustomer_sale_values = ICustomerSale.objects.filter(icustomer_sale_group__in=icustomer_sale_group,
                                                         product__is_active=True).values_list(
        'product_id', 'count')
    icustomer_sale_columns = ['product_id', 'count']
    icustomer_sale_df = pd.DataFrame(list(icustomer_sale_values), columns=icustomer_sale_columns)

    combined_df = sale_df.append(icustomer_sale_df)
    product_values = Product.objects.filter(is_active=True).order_by('display_ordinal').values_list('id', 'short_name',
                                                                                                    'unit', 'quantity')
    product_columns = ['id', 'product_short_name', 'unit', 'quantity']
    product_df = pd.DataFrame(list(product_values), columns=product_columns)
    total_sale_df = combined_df.groupby(['product_id'])['count'].sum().reset_index()
    merged_df = pd.merge(total_sale_df, product_df, how='left', left_on='product_id', right_on='id')
    tray_config_values = ProductTrayConfig.objects.filter(product__is_active=True).values_list('product_id',
                                                                                               'tray_count',
                                                                                               'product_count')
    tray_config_columns = ['product_id', 'p_c_tray_count', 'p_c_product_count']
    tray_config_df = pd.DataFrame(list(tray_config_values), columns=tray_config_columns)
    tray_config_merge_df = pd.merge(merged_df, tray_config_df, how='left', left_on='id', right_on='product_id')
    # finding the tray count based on the defalut product tray count
    tray_config_merge_df['calculated_tray_count'] = tray_config_merge_df.apply(
        lambda x: findout_tray_count(x['count'], x['p_c_product_count'], x['id']), axis=1)
    tray_config_merge_df = tray_config_merge_df.fillna(0)
    # print(tray_config_merge_df['calculated_tray_count'])
    tray_config_merge_df = tray_config_merge_df[(tray_config_merge_df['id'] != 10) & (tray_config_merge_df['id'] != 26) & (tray_config_merge_df['id'] != 28)]
    print(tray_config_merge_df)
    
    return tray_config_merge_df['calculated_tray_count'].sum()

In [20]:
import datetime
date = datetime.datetime.now().date()
date += datetime.timedelta(days=1)
check_tray_count(94, date, 1)

   product_id_x  count  id product_short_name  unit  quantity  product_id_y  \
0             1  219.0   1             TM 500     1   500.000             1   
1             2  317.0   2            STD 500     1   500.000             2   
2             3  114.0   3            STD 250     1   250.000             3   
3             4   81.0   4            FCM 500     1   500.000             4   
4             6    1.0   6           FCM 1000     1  1000.000             6   
5             8   12.0   8             BM 200     1   200.000             8   

   p_c_tray_count  p_c_product_count  calculated_tray_count  
0               1                 24                    9.0  
1               1                 24                   13.0  
2               1                 48                    2.0  
3               1                 24                    3.0  
4               1                 12                    0.0  
5               1                 40                    0.0  


27.0

In [27]:
def check_next_day_order(counter_id, date, expected_date):
    employee_trace_ids = list(CounterEmployeeTraceMap.objects.filter(collection_date=date, counter_id=counter_id).values_list('id', flat=True))
    sale_group_ids = list(CounterEmployeeTraceSaleGroupMap.objects.filter(counter_employee_trace_id__in=employee_trace_ids).values_list('sale_group', flat=True))
    sale_group_obj = SaleGroup.objects.filter(id__in=sale_group_ids)
    unique_date = list(set(sale_group_obj.values_list('date', flat=True)))
    total_amount = sale_group_obj.aggregate(Sum('total_cost'))
    print(f"Total amount ordered - {total_amount}")
    print(f"Delivery Date list - {unique_date}")
    print('----------------')
    other_date_order_obj = sale_group_obj.exclude(date=expected_date)
    for order in other_date_order_obj:
        print(f"Total cost - {order.total_cost}, Booth code - {order.business.code}, Delivery date - {order.date}, Session - {order.session.name}")
    
check_next_day_order(3, '2021-04-12', '2021-04-13')

Total amount ordered - {'total_cost__sum': Decimal('1373752.80')}
Delivery Date list - [datetime.date(2021, 4, 13), datetime.date(2021, 4, 14)]
----------------
Total cost - 2366.50, Booth code - 2697, Delivery date - 2021-04-14, Session - Morning
Total cost - 1078.00, Booth code - 2697, Delivery date - 2021-04-14, Session - Evening


In [23]:
check_next_day_order(3, '2021-04-12', '2021-04-13')

2366.50 2697 2021-04-14
1078.00 2697 2021-04-14
{'total_cost__sum': Decimal('1373752.80')}
[datetime.date(2021, 4, 13), datetime.date(2021, 4, 14)]
